### Configuración

In [ ]:
# @title Dependencias necesarias
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import folium

In [ ]:
# @title Declaración de métodos y constantes
# Ideas sacadas de https://www.jpytr.com/post/analysinggeographicdatawithfolium/

colors = [
    'red',
    'blue',
    'gray',
    'darkred',
    'lightred',
    'orange',
    'beige',
    'green',
    'darkgreen',
    'lightgreen',
    'darkblue',
    'lightblue',
    'purple',
    'darkpurple',
    'pink',
    'cadetblue',
    'lightgray',
    'black'
]

def dibujar_mapa():
    mapa = folium.Map(location=(-31.30, -64.12),
                      zoom_start=11)
    return mapa


def dibujar_grilla(mapa, grilla):
    for i, celda in grilla.iterrows():
        upper_left = (celda["lower_left_lng"], celda["upper_right_lat"])
        upper_right = (celda["upper_right_lng"], celda["upper_right_lat"])
        lower_right = (celda["upper_right_lng"], celda["lower_left_lat"])
        lower_left = (celda["lower_left_lng"], celda["lower_left_lat"])

        coordinates = [
            upper_left,
            upper_right,
            lower_right,
            lower_left,
            upper_left
        ]
        geo_json = {"type": "FeatureCollection",
                    "properties": {
                        "name": celda["name"],
                    },
                    "features": [{
                        "type": "Feature",
                        "geometry": {
                            "type": "Polygon",
                            "coordinates": [coordinates],
                        }
                    }]
                   }

        color = plt.cm.Reds(i / len(grilla))
        color = mpl.colors.to_hex(color)

        gj = folium.GeoJson(geo_json,
                            style_function=lambda feature, color=color: {'fillColor': color,
                                                                         'color': "black",
                                                                         'weight': 2,
                                                                         'fillOpacity': 0.15,
                                                                         'dashArray': '5, 5',
                                                                         })
        popup = folium.Popup(geo_json["properties"]["name"])
        gj.add_child(popup)

        mapa.add_child(gj)

    return mapa


def dibujar_recorrido(mapa, recorrido, color="blue"):
    folium.PolyLine(recorrido, color=color, weight=2.5, opacity=1).add_to(mapa)
    for lat, lng in recorrido:
        folium.Marker(location=[lat, lng], icon=folium.Icon(color=color)).add_to(mapa)
    return mapa

### Definimos los archivos con los datos
**Importante:** subirlos al Colab (panel izquierdo) y asignar a las variables los nombres correspondientes

In [ ]:
grilla_filename     = 'grilla_ejemplo.csv'
recorridos_filename = 'recorridos_ejemplo.csv'

### Dibujamos!

In [ ]:
mapa = dibujar_mapa()

grilla = pd.read_csv(grilla_filename, sep="\t", names=["lower_left_lat",  "lower_left_lng", "upper_right_lat", "upper_right_lng", "name"], index_col=None)
mapa   = dibujar_grilla(mapa, grilla)

recorridos = pd.read_csv(recorridos_filename, sep="\t", names=["recorrido_id", "lat", "lng"], index_col=None)
for i, rec in enumerate(recorridos.groupby("recorrido_id").apply(lambda r: list(zip(r.lat, r.lng)))):
  mapa = dibujar_recorrido(mapa, rec, color=colors[i])

mapa